## Практическая работа №2. Обработка текстовых данных с помощью байесовской классификации

Реализовать алгоритм байесовской классификации (NaiveBayes) для решения задачи классификации текстовых сообщений (spam/ham).

Оценить точность классификации на обучающем и на тестовом множестве при различной предварительной обработке текстовых данных: 
1. по всем словам без учета регистра, 
2. по всем словам с учетом регистра,
3. только по специфическим словам.

Каждое сообщение необходимо разбить на отдельные слова (признаки). 
При этом:
1) можно учитывать или не учитывать регистры слов ("NOW" == "now");
2) учитывать только часто встречающиеся слова для spam-сообщений (topWordsSpam) и ham-сообщений (topWordsHam);
3) учитывать только специфические слова (входящие либо в topWordsSpam, либо в topWordsHam).

При разной предварительной обработке можно получить разные оценки точности классификации.

In [51]:
import pandas as pd
import numpy as np
import math
import re
import math
import string as pString


data = pd.read_csv("./spam.csv", sep=',', usecols =[0,1], names=['class','sms'], skiprows=[0], encoding='latin-1')
data

,class,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [52]:
#Подсчёт сколько раз слова встречались в данном классе
def TopWordsCounter(data: pd.DataFrame, className = 'spam', need_punct=False, need_register=False):
    topWords = dict()
    #Добавляем пунктуацию в словарь
    if need_punct:
        for string in data.itertuples():
            tempPunct = set(string[2])&set(pString.punctuation)
            for symbol in tempPunct:
                repeatCount = string[2].count(symbol)
                if symbol in topWords.keys():
                    topWords[symbol] += repeatCount
                else:
                    topWords[symbol] = repeatCount
    
    #Проходимся по строкам и выделяем слова                
    for string in data.itertuples():
        regPunctuation = '\s|\r\n|\r|\n|\[|\"|\||\}|#|\.|\+|\(|\?|-|;|{|<|:|\\|\)|`|_|\*|~|/|\@|,|%|=|\'|\^|>|&|\$|!'
        tempWords = re.split(regPunctuation, string[2])
        for word in tempWords:
            #Убираем слова длиной 1 или меньше
            if len(word) <= 1:
                continue;
            #Приводим слова к нижнему регистру, если не нужен его учёт
            if not need_register:
                word = word.lower()
            if word in topWords.keys():
                topWords[word] += 1
            else: 
                topWords[word] = 1
    return topWords

def Classify(testData, trainData, features):
    desicions = {}
    #Для каждой тестовой строки    
    for testString in testData.itertuples():
        testMessage = testString[2]
        #V - Кол-во уникальных слов любых классов в обучающей выборке
        V = len(features)
        #Проходимся по каждому классу
        classScores = [0 for i in range(len(features.columns))]
        for i in range(len(features.columns)):
            #Кол-во уникальных слов данного класса в обучающей выборке
            Lc = trainData.iloc[:,i].astype(bool).sum()
            
            #Dc - кол-во сообщений в обучающей выборке принадлежащих классу
            Dc = trainData.iloc[:,0].value_counts()[features.columns[i].lower()]
            #D - общее кол-во сообщений в обучающей выборке
            D = float(len(trainData))
            classScores[i] = math.log(Dc / D)

            #Проходимся по каждому слову в сообщении
            regPunctuation = '\s|\r\n|\r|\n|\[|\"|\||\}|#|\.|\+|\(|\?|-|;|{|<|:|\\|\)|`|_|\*|~|/|\@|,|%|=|\'|\^|>|&|\$|!'
            for word in re.split(regPunctuation, testMessage):
                if word in features.index:
                    #сколько раз слово встречалось в сообщениях класса в обучающей выборке
                    Wc = features.loc[word][i]
                else: 
                    Wc = 0
                
                classScores[i] += math.log((Wc + 1) / (V + Lc))
        index_max = max(range(len(classScores)), key=classScores.__getitem__)
        desicions.update({testString[0]: features.columns[index_max]})
    return desicions

#### ► Задаём параметры и запускаем классификацию

In [53]:
#PARAMETERS
need_punct = True
need_register = True
need_only_spec_words = True
need_Classify_on_TrainData = False

#Разделим исходные данные на обучающую и тестовую выборки
#frac - доля данных, которую берём от исходных
#.sample(frac=1) - перемешивание строк
tempData = data.sample(frac=1)
trainData = tempData.sample(frac=0.66, random_state=1)
if need_Classify_on_TrainData: testData=trainData
else: testData = tempData.drop(trainData.index).sample(frac=1)


trainData.head()

topWordsSpam = TopWordsCounter(trainData.loc[trainData['class'] == 'spam'], 'spam', need_punct, need_register)
topWordsHam = TopWordsCounter(trainData.loc[trainData['class'] == 'ham'], 'ham', need_punct, need_register)

topWordsSpam = pd.DataFrame.from_dict(topWordsSpam, orient='index')
topWordsHam = pd.DataFrame.from_dict(topWordsHam, orient='index')
topWordsTable = pd.concat([topWordsSpam, topWordsHam],axis=1,join='outer')
topWordsTable.fillna(0, inplace = True)
topWordsTable.columns = ['spam','ham']
topWordsTable = topWordsTable.astype(int)
topWordsTable

,spam,ham
.,1030,6350
:,116,380
-,180,221
",",275,1011
+,70,26
...,...,...
\Life,0,1
everything\,0,1
Real,0,1
absence,0,1


In [54]:
print(f"Всего слов: {len(topWordsTable)}")
#Если учитывать только специфические слова - те, что встречаются только в одном классе
if need_only_spec_words:
    tempTable = []
    for x in topWordsTable.itertuples():
        if min(x[1:]) == 0:        
                tempTable.append(x[0])
    topWordsTable = topWordsTable.loc[tempTable]
print(f"Из них СПЕЦИФИЧЕСКИХ слов: {len(topWordsTable)}")

desicions = Classify(testData, trainData, topWordsTable)

Всего слов: 8818
Из них СПЕЦИФИЧЕСКИХ слов: 7902


In [55]:
#Блок подсчёта точности
TP, TN, FP, FN = 0, 0, 0, 0
    
#SPAM Class Statistics
for row_index in desicions.keys():
    if desicions[row_index] == testData.loc[row_index][0] and desicions[row_index] == 'spam':
        TP += 1
    elif desicions[row_index] == testData.loc[row_index][0] and desicions[row_index] == 'ham':
        TN += 1
    elif desicions[row_index] != testData.loc[row_index][0] and desicions[row_index] == 'spam':
        FP += 1
    elif desicions[row_index] != testData.loc[row_index][0] and desicions[row_index] == 'ham':
        FN += 1

    
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
beta = 1
f1_score = (1+math.pow(beta,2)) * (precision * recall) / (math.pow(beta, 2) * precision + recall)

if need_register: strREG = "ЕСТЬ регистр"
else: strREG = "НЕТ регистра"
if need_punct: strPUN = "ЗНАКИ препинания"
else: strPUN = "БЕЗ знаков"
if need_only_spec_words: strNOSW = "Только СПЕЦИФИЧЕСКИЕ слова"
else: strNOSW = "ВСЕ слова"
if need_Classify_on_TrainData: print("Классификация TRAIN DATA")
else: print("Классификация на TEST DATA")

print(f"{strREG} / {strPUN} / {strNOSW}")
print(f"Accuracy = {accuracy} \nPrecision = {precision} \nRecall = {recall} \nF1_Score = {f1_score}")  

Классификация на TEST DATA
ЕСТЬ регистр / ЗНАКИ препинания / Только СПЕЦИФИЧЕСКИЕ слова
Accuracy = 0.9751847940865892 
Precision = 0.9950738916256158 
Recall = 0.8145161290322581 
F1_Score = 0.8957871396895788
